[![Abrir en Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pugapatricia/gestion-documentaria-para-pymes/blob/main/etiquetado/Generador_etiquetas.ipynb)

[![Ver en GitHub](https://img.shields.io/badge/GitHub-Repo-black?logo=github)](https://github.com/pugapatricia/gestion-documentaria-para-pymes/tree/main/etiquetado)

#Importaciones

In [17]:
pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install requests msal python-docx PyPDF2 pandas python-pptx torch openai python-dotenv


Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import msal
from docx import Document
import PyPDF2
import pandas as pd
from pptx import Presentation
import os
import torch
from openai import OpenAI
import re
from getpass import getpass
from dotenv import load_dotenv

# Configuración

In [10]:
import os
from dotenv import load_dotenv

# Ruta relativa desde este notebook
dotenv_path = "../password.env"  # sube un nivel desde scripts
load_dotenv(dotenv_path)

api_key = os.getenv("api_key")
CLIENT_ID = os.getenv("CLIENT_ID")
token = os.getenv("token")

In [13]:
AUTHORITY = "https://login.microsoftonline.com/consumers"
SCOPE = ["User.Read", "Files.ReadWrite"]
ext_permitidas = {"pdf", "docx", "xlsx", "xls", "pptx", "txt", "csv"}
url = "https://graph.microsoft.com/v1.0/me/drive/root:/Etiquetados:/children"


In [ ]:
!git config --global user.email "marcomendieta08@gmail.com"
!git config --global user.name "marcomendieta08"
!git clone https://github.com/pugapatricia/gestion-documentaria-para-pymes.git


# Conección con OneDrive

In [ ]:
app = msal.PublicClientApplication(CLIENT_ID, authority=AUTHORITY)

flow = app.initiate_device_flow(scopes=SCOPE)
if "user_code" not in flow:
    raise Exception("No se pudo iniciar el device flow. Revisa tu configuración en Azure.")

print(flow["message"])  # 👉 Copia el código en https://microsoft.com/devicelogin
result = app.acquire_token_by_device_flow(flow)

if "access_token" not in result:
    raise Exception(f"Error autenticación: {result.get('error_description')}")

access_token = result["access_token"]
headers = {"Authorization": f"Bearer {access_token}"}

# Llamada a la API con tu token de acceso
resp = requests.get(url, headers=headers)
if resp.status_code != 200:
    raise Exception(f"Error al obtener archivos: {resp.text}")
data = resp.json()


# Funciones

Procesamos cada documento de acuerdo con su tipo (PDF, Word, Excel, etc.) para convertirlo en datos estructurados. Este proceso se realiza mediante las siguientes funciones.

In [ ]:
def leer_pdf(file_path):
    text = ""
    with open(file_path, 'rb') as f:
        reader = PyPDF2.PdfReader(f)
        for page in reader.pages:
            text += page.extract_text() or ""
    return text

def leer_docx(file_path):
    doc = Document(file_path)
    return "\n".join([p.text for p in doc.paragraphs])

def leer_excel(file_path):
    try:
        df_dict = pd.read_excel(file_path, sheet_name=None)
    except Exception as e:
        return f"Error leyendo Excel: {e}"

    texto = []
    for nombre, hoja in df_dict.items():
        hoja = hoja.fillna("")  # reemplaza NaN por ""
        texto.append(f"\n--- Hoja: {nombre} ---\n")
        texto.append(hoja.to_string())
    return "\n".join(texto)

def leer_csv(file_path):
    df = pd.read_csv(file_path)
    return df.to_string()

def leer_pptx(file_path):
    prs = Presentation(file_path)
    texto = []
    for i, slide in enumerate(prs.slides, 1):
        texto.append(f"\n--- Diapositiva {i} ---\n")
        for shape in slide.shapes:
            if hasattr(shape, "text_frame") and shape.text_frame:
                texto.append(shape.text_frame.text)
    return "\n".join(texto)

def leer_txt(file_path):
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        return f.read()

Función para solicitar a OpenAI la generación de etiquetas/tickers por documento, con un límite máximo

In [ ]:
etiquetas_global = set()

def sugerir_tickers(texto, max_etiquetas=10):
    if not texto.strip():
        return []

    prompt = f"""
Eres un asistente que recibe un texto de un documento.
Devuelve solo las {max_etiquetas} palabras más importantes
que podrían usarse como etiquetas del documento, una sola palabra cada una,
en una lista separada por comas. No agregues explicaciones, solo las palabras.

Texto:
{texto}
"""
    try:
        respuesta = client.chat.completions.create(
            model="gpt-5-mini",
            messages=[{"role": "user", "content": prompt}],
        )

        etiquetas = respuesta.choices[0].message.content
        etiquetas_lista = [e.strip().lower() for e in re.split(r'[,\n;]+', etiquetas) if e.strip()]
        etiquetas_lista = list(dict.fromkeys(etiquetas_lista))[:max_etiquetas]
        etiquetas_global.update(etiquetas_lista)

        return etiquetas_lista

    except Exception as e:
        print(f"⚠️ Error al generar etiquetas: {e}")
        return []


Descarga un archivo temporal desde una URL, verifica que su extensión esté permitida y lo lee según su tipo, devolviendo su contenido como texto.

In [ ]:
tickers_global = set()
ext_permitidas = {"pdf", "docx", "xlsx", "xls", "pptx", "txt", "csv"}

def leer_archivo(item):
    nombre = item.get("name")
    ext = nombre.split(".")[-1].lower()
    if ext not in ext_permitidas:
        return ""

    tmp_path = f"/tmp/{nombre}"
    try:
        r = requests.get(item["@microsoft.graph.downloadUrl"], stream=True)
        r.raise_for_status()
        with open(tmp_path, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)

        if ext == "pdf":
            return leer_pdf(tmp_path)
        elif ext == "docx":
            return leer_docx(tmp_path)
        elif ext in {"xlsx", "xls"}:
            return leer_excel(tmp_path)
        elif ext == "pptx":
            return leer_pptx(tmp_path)
        elif ext == "txt":
            return leer_txt(tmp_path)
        elif ext == "csv":
            return leer_csv(tmp_path)
    except requests.RequestException as e:
        print(f"⚠️ Error descargando {nombre}: {e}")
    except Exception as e:
        print(f"⚠️ Error procesando {nombre}: {e}")
    finally:
        if os.path.exists(tmp_path):
            os.remove(tmp_path)

    return ""


# LISTAR Y PROCESAR ARCHIVOS

In [ ]:
etiquetas = set()

for item in data.get("value", []):
    if "folder" in item:
        continue

    texto = leer_archivo(item)
    if texto.strip():
        try:
            sugerencias = sugerir_tickers(texto, max_etiquetas=10)
            etiquetas.update([s.strip() for s in sugerencias if s.strip()])
        except Exception as e:
            print(f"⚠️ Error generando etiquetas para {item.get('name')}: {e}")

print("\nTickers finales:\n", sorted(etiquetas))
len(etiquetas)

In [ ]:
with open("etiquetado/tickers.txt", "w") as f:
    f.write(", ".join(etiquetas))

In [ ]:
!git add -u etiquetado/tickers.txt
!git diff --cached --quiet || git commit -m "Actualizar tickers.txt desde Colab"
!git pull https://{token}@github.com/pugapatricia/gestion-documentaria-para-pymes.git main
!git push https://{token}@github.com/pugapatricia/gestion-documentaria-para-pymes.git main